In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")

print("Libraries loaded successfully!")
print("Starting Customer Transaction Analysis...")


Libraries loaded successfully!
Starting Customer Transaction Analysis...


In [2]:
# Load the data
print("Loading transaction and customer data...")

# Load transaction data
transactions_df = pd.read_csv('../NewData/sales_transactions.csv')
customers_df = pd.read_csv('../NewData/customers.csv')

print(f"Transactions data shape: {transactions_df.shape}")
print(f"Customers data shape: {customers_df.shape}")

# Display basic info about the datasets
print("\n=== TRANSACTION DATA INFO ===")
print(transactions_df.info())
print("\nFirst few rows:")
print(transactions_df.head())

print("\n=== CUSTOMER DATA INFO ===")
print(customers_df.info())
print("\nFirst few rows:")
print(customers_df.head())


Loading transaction and customer data...
Transactions data shape: (7949, 7)
Customers data shape: (300, 7)

=== TRANSACTION DATA INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7949 entries, 0 to 7948
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    7949 non-null   object 
 1   transaction_date  7949 non-null   object 
 2   customer_id       7949 non-null   object 
 3   store_id          7949 non-null   object 
 4   product_id        7949 non-null   object 
 5   quantity          7949 non-null   int64  
 6   total_amount      7949 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 434.8+ KB
None

First few rows:
  transaction_id transaction_date customer_id store_id product_id  quantity  \
0         T02579       2024-04-21       C0001     S001      P0142         3   
1         T02579       2024-04-21       C0001     S001      P0055         2   
2         

In [3]:
# Data preprocessing and analysis
print("=== DATA PREPROCESSING ===")

# Convert transaction_date to datetime
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'])

# Check for missing values
print("Missing values in transactions:")
print(transactions_df.isnull().sum())

print("\nMissing values in customers:")
print(customers_df.isnull().sum())

# Check date range
print(f"\nTransaction date range: {transactions_df['transaction_date'].min()} to {transactions_df['transaction_date'].max()}")

# Check unique customers in transactions vs customer master data
unique_customers_in_transactions = transactions_df['customer_id'].nunique()
unique_customers_in_master = customers_df['customer_id'].nunique()

print(f"\nUnique customers in transactions: {unique_customers_in_transactions}")
print(f"Unique customers in master data: {unique_customers_in_master}")

# Check if all transaction customers exist in customer master
customers_in_trans_not_in_master = set(transactions_df['customer_id'].unique()) - set(customers_df['customer_id'].unique())
print(f"Customers in transactions but not in master data: {len(customers_in_trans_not_in_master)}")
if len(customers_in_trans_not_in_master) > 0:
    print("Sample missing customers:", list(customers_in_trans_not_in_master)[:5])


=== DATA PREPROCESSING ===
Missing values in transactions:
transaction_id      0
transaction_date    0
customer_id         0
store_id            0
product_id          0
quantity            0
total_amount        0
dtype: int64

Missing values in customers:
customer_id     0
name            0
age             0
gender          0
city            0
vehicle_type    0
vehicle_name    0
dtype: int64

Transaction date range: 2022-01-01 00:00:00 to 2024-12-30 00:00:00

Unique customers in transactions: 300
Unique customers in master data: 300
Customers in transactions but not in master data: 0


In [4]:
# CORE ANALYSIS: Transaction Count and Last Transaction Date per Customer
print("=== CUSTOMER TRANSACTION ANALYSIS ===")

# Calculate transaction counts per customer (based on unique transaction_id per customer)
customer_transaction_summary = transactions_df.groupby('customer_id').agg({
    'transaction_id': 'nunique',  # Count unique transactions
    'transaction_date': ['min', 'max'],  # First and last transaction dates
    'total_amount': ['sum', 'mean'],  # Total spending and average per transaction
    'quantity': 'sum'  # Total items purchased
}).round(2)

# Flatten column names
customer_transaction_summary.columns = [
    'total_transactions', 
    'first_transaction_date', 
    'last_transaction_date',
    'total_spending',
    'avg_spending_per_transaction',
    'total_items_purchased'
]

# Reset index to make customer_id a column
customer_transaction_summary = customer_transaction_summary.reset_index()

# Merge with customer information
customer_analysis = customer_transaction_summary.merge(
    customers_df[['customer_id', 'name', 'age', 'gender', 'city']], 
    on='customer_id', 
    how='left'
)

print(f"Analysis completed for {len(customer_analysis)} customers")
print("\nSample of customer transaction analysis:")
print(customer_analysis.head(10))

# Display summary statistics
print("\n=== SUMMARY STATISTICS ===")
print("Transaction count distribution:")
print(customer_analysis['total_transactions'].describe())

print(f"\nDate range analysis:")
print(f"Earliest first transaction: {customer_analysis['first_transaction_date'].min()}")
print(f"Latest first transaction: {customer_analysis['first_transaction_date'].max()}")
print(f"Earliest last transaction: {customer_analysis['last_transaction_date'].min()}")
print(f"Latest last transaction: {customer_analysis['last_transaction_date'].max()}")


=== CUSTOMER TRANSACTION ANALYSIS ===
Analysis completed for 300 customers

Sample of customer transaction analysis:
  customer_id  total_transactions first_transaction_date  \
0       C0001                   8             2022-05-23   
1       C0002                   6             2022-03-03   
2       C0003                   6             2022-02-15   
3       C0004                  11             2022-01-15   
4       C0005                  17             2022-01-27   
5       C0006                   9             2022-07-20   
6       C0007                  11             2022-12-02   
7       C0008                   6             2022-02-09   
8       C0009                  12             2022-01-01   
9       C0010                   5             2022-05-11   

  last_transaction_date  total_spending  avg_spending_per_transaction  \
0            2024-10-01         9628.17                        566.36   
1            2024-07-05         8336.02                        490.35   
2  

In [5]:
# ANALYSIS: Gap between Last and Second-to-Last Transaction
print("=== TRANSACTION GAP ANALYSIS ===")

# Get the last two transaction dates for each customer
customer_transaction_gaps = []

for customer_id in transactions_df['customer_id'].unique():
    customer_transactions = transactions_df[transactions_df['customer_id'] == customer_id]
    
    # Get unique transaction dates sorted in descending order
    unique_dates = customer_transactions['transaction_date'].drop_duplicates().sort_values(ascending=False)
    
    if len(unique_dates) >= 2:
        last_transaction = unique_dates.iloc[0]
        second_last_transaction = unique_dates.iloc[1]
        gap_days = (last_transaction - second_last_transaction).days
        
        customer_transaction_gaps.append({
            'customer_id': customer_id,
            'last_transaction_date': last_transaction,
            'second_last_transaction_date': second_last_transaction,
            'gap_days': gap_days
        })
    else:
        # Customer has only one transaction
        customer_transaction_gaps.append({
            'customer_id': customer_id,
            'last_transaction_date': unique_dates.iloc[0] if len(unique_dates) > 0 else None,
            'second_last_transaction_date': None,
            'gap_days': None
        })

# Convert to DataFrame
transaction_gaps_df = pd.DataFrame(customer_transaction_gaps)

# Display basic statistics
print(f"Total customers analyzed: {len(transaction_gaps_df)}")
print(f"Customers with multiple transactions: {len(transaction_gaps_df[transaction_gaps_df['gap_days'].notna()])}")
print(f"Customers with only one transaction: {len(transaction_gaps_df[transaction_gaps_df['gap_days'].isna()])}")

# Focus on customers with multiple transactions
multi_transaction_customers = transaction_gaps_df[transaction_gaps_df['gap_days'].notna()].copy()

if len(multi_transaction_customers) > 0:
    print(f"\n=== GAP STATISTICS (for customers with multiple transactions) ===")
    print("Gap between last and second-to-last transaction (days):")
    print(multi_transaction_customers['gap_days'].describe())
    
    # Count customers with gaps over 180 days
    customers_over_180_days = multi_transaction_customers[multi_transaction_customers['gap_days'] > 180]
    count_over_180_days = len(customers_over_180_days)
    
    print(f"\n=== KEY FINDINGS ===")
    print(f"Customers with gaps over 180 days: {count_over_180_days}")
    print(f"Percentage of multi-transaction customers with gaps over 180 days: {count_over_180_days/len(multi_transaction_customers)*100:.2f}%")
    
    # Show some examples of customers with large gaps
    if count_over_180_days > 0:
        print(f"\nTop 10 customers with largest gaps:")
        top_gaps = customers_over_180_days.nlargest(10, 'gap_days')[['customer_id', 'gap_days', 'last_transaction_date', 'second_last_transaction_date']]
        print(top_gaps.to_string(index=False))
        
        # Distribution of gaps
        print(f"\nGap distribution for customers with gaps over 180 days:")
        gap_ranges = [
            ('over 3 months', (multi_transaction_customers['gap_days'] > 90)),
            ('over 4 months', (multi_transaction_customers['gap_days'] > 120)),
            ('over 5 months', (multi_transaction_customers['gap_days'] > 150)),
            ('over 6 months', (multi_transaction_customers['gap_days'] > 180))
        ]
        
        for range_name, condition in gap_ranges:
            count = len(multi_transaction_customers[condition])
            print(f"  {range_name}: {count} customers")
    
else:
    print("No customers found with multiple transactions to analyze gaps.")

# Save the gap analysis results
gap_output_file = '../NewData/customer_transaction_gaps.csv'
transaction_gaps_df.to_csv(gap_output_file, index=False)
print(f"\nTransaction gap analysis saved to: {gap_output_file}")


=== TRANSACTION GAP ANALYSIS ===
Total customers analyzed: 300
Customers with multiple transactions: 300
Customers with only one transaction: 0

=== GAP STATISTICS (for customers with multiple transactions) ===
Gap between last and second-to-last transaction (days):
count    300.000000
mean      99.973333
std       96.259331
min        1.000000
25%       35.000000
50%       67.000000
75%      131.000000
max      573.000000
Name: gap_days, dtype: float64

=== KEY FINDINGS ===
Customers with gaps over 180 days: 49
Percentage of multi-transaction customers with gaps over 180 days: 16.33%

Top 10 customers with largest gaps:
customer_id  gap_days last_transaction_date second_last_transaction_date
      C0090       573            2024-12-03                   2023-05-10
      C0247       543            2024-09-26                   2023-04-02
      C0273       490            2024-11-18                   2023-07-17
      C0216       456            2024-10-21                   2023-07-23
      